In [1]:
# 필요한 라이브러리를 임포트하기

import sys # 시스템
import os  # 시스템

# 데이터 다루기
import pandas as pd
import numpy as np

# selenium 크롤링
from selenium import webdriver  
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 크롬 드라이버
import chromedriver_autoinstaller

# beautifulsoup 크롤링
import requests
from bs4 import BeautifulSoup

# lxml 크롤링
import lxml.html

# 시간 조절
import time

# 시간 측정
from tqdm import notebook

# 정규표현식
import re

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# 키워드 URL 검색용으로 변환
from urllib import parse

In [2]:
# 지역 키워드를 입력하기
keyword = '한양대 근처'
keyword = parse.quote(keyword)

# page 정하기
page = 1

In [3]:
# 크롬창 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
driver.maximize_window() # 창 최대화

# 키워드가 입력된 망고플레이트 사이트에 들어가기
driver.get("https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, page)) 
time.sleep(2)

In [5]:
"https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, page)

'https://www.mangoplate.com/search/%ED%95%9C%EC%96%91%EB%8C%80%20%EA%B7%BC%EC%B2%98?keyword=%ED%95%9C%EC%96%91%EB%8C%80%20%EA%B7%BC%EC%B2%98&page=1'

In [6]:
# 첫 번째 음식점 클릭
element = 'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child(1) > div:nth-child(2) > figure > a > div > img'
driver.find_element(By.CSS_SELECTOR, element).click()

In [7]:
# 가게 이름 크롤링
element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1'
title_raw = driver.find_element(By.CSS_SELECTOR, element)
title = title_raw.text
print(title, type(title))

모음 <class 'str'>


In [8]:
# 가게 전체 평점 점수 크롤링
element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
total_raw = driver.find_element(By.CSS_SELECTOR, element)
total = total_raw.text
print(total, type(total))

4.3 <class 'str'>


In [9]:
driver.back()

In [2]:
# 음식점 1개에서 정보 크롤링 함수
def getFoodInfo(driver, url):    
    # 키워드가 입력된 망고플레이트 사이트에 들어가기
    driver.get(url)
    cur_url = driver.current_url
    sub_dict = {}

    # 가게 이름 크롤링
    element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > h1'
    title_raw = driver.find_element(By.CSS_SELECTOR, element)
    title = title_raw.text

    # 가게 전체 평점 점수 크롤링
    try:
        element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.restaurant_title_wrap > span > strong > span'
        total_raw = driver.find_element(By.CSS_SELECTOR, element)
        total = float(total_raw.text)
    except:
        total = 0.

    # 조회수 크롤링
    try:
        element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.hit'
        view_raw = driver.find_element(By.CSS_SELECTOR, element)
        view = view_raw.text
    except:
        element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.hit'
        view_raw = driver.find_element(By.CSS_SELECTOR, element)
        view = view_raw.text

    # 리뷰 개수 크롤링
    try:
        element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.review'
        review_raw = driver.find_element(By.CSS_SELECTOR, element)
        num_review = review_raw.text

    except:
        element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.review'
        review_raw = driver.find_element(By.CSS_SELECTOR, element)
        num_review = review_raw.text

    # 별표 개수 크롤링
    try:
        element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status.branch_none > span.cnt.favorite'
        star_raw = driver.find_element(By.CSS_SELECTOR, element)
        num_star = star_raw.text

    except:
        element = 'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > header > div.status > span.cnt.favorite'
        star_raw = driver.find_element(By.CSS_SELECTOR, element)
        num_star = star_raw.text
    
    # 가격대 크롤링
    price = ""
    for idx in range(10):
        try:
            tr_head = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({idx+1}) > th'
            head_raw = driver.find_element(By.CSS_SELECTOR, tr_head)
            head = head_raw.text
            if '가격대' == head.strip():
                element1 = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({idx+1}) > td'
                type_raw = driver.find_element(By.CSS_SELECTOR, element1)
                type_raw = type_raw.text
                price = type_raw
                break
        except:
            pass

    # 영업 시간 크롤링
    open_hour = ""
    breaktime = ""
    closed_day = ""
    for idx in range(10):
        try:
            tr_head = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({idx+1}) > th'
            head_raw = driver.find_element(By.CSS_SELECTOR, tr_head)
            head = head_raw.text
            if '영업시간' == head.strip():
                element1 = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({idx+1}) > td'
                type_raw = driver.find_element(By.CSS_SELECTOR, element1)
                type_raw = type_raw.text
                open_hour = type_raw
            if '쉬는시간' == head.strip():
                element1 = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({idx+1}) > td'
                type_raw = driver.find_element(By.CSS_SELECTOR, element1)
                type_raw = type_raw.text
                breaktime = type_raw
            if '휴일' == head.strip():
                element1 = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({idx+1}) > td'
                type_raw = driver.find_element(By.CSS_SELECTOR, element1)
                type_raw = type_raw.text
                closed_day = type_raw
        except:
            pass
        
    # 음식 종류 및 메뉴 크롤링
    for idx in range(10):
        try:
            tr_head = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({idx+1}) > th'
            head_raw = driver.find_element(By.CSS_SELECTOR, tr_head)
            head = head_raw.text
            if '음식 종류' == head.strip():
                element1 = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({idx+1}) > td'
                type_raw = driver.find_element(By.CSS_SELECTOR, element1)
                type_raw = type_raw.text
                food_type = [type_raw, []]
                break
        except:
            pass
    
    is_menu_available = False
    for idx in range(10):
        try:
            tr_head = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({idx+1}) > th'
            head_raw = driver.find_element(By.CSS_SELECTOR, tr_head)
            head = head_raw.text
            if '메뉴' in head:
                is_menu_available = True
                break
        except:
            pass

    if is_menu_available:
        try:
            element2 = f'body > main > article > div.column-wrapper > div.column-contents > div > section.restaurant-detail > table > tbody > tr:nth-child({idx+1}) > td > ul'
            menu_raw = driver.find_element(By.CSS_SELECTOR, element2)
            food_menu = menu_raw.text
            food_menu = food_menu.split('\n')
            food_menu = [(food_menu[2*i], food_menu[2*i+1]) for i in range(int(len(food_menu)/2))]
            food_type = [type_raw, food_menu]
        except:
            pass

    # sub_dict에 담기
    sub_dict['title'] = title
    sub_dict['Point'] = total
    sub_dict['View'] = view
    sub_dict['Review'] = num_review
    sub_dict['Star'] = num_star
    sub_dict["Open_Hour"] = open_hour
    sub_dict["Breaktime"] = breaktime
    sub_dict["Closed_Day"] = closed_day
    sub_dict["Price"] = price
    sub_dict['Type'] = food_type[0]
    sub_dict["Menu"] = food_type[1]
    sub_dict['URL'] = cur_url
            
    return sub_dict

In [3]:
# 키워드에 해당하는 음식점 리스트 탐색
def getFoodList(driver, keyword, page=1, row=10, col=2):
    # 전체 담을 그릇
    total_dict = {}
    k = 0
    
    keyword = parse.quote(keyword)
    
    for p in range(1, page+1):
        # 키워드가 입력된 망고플레이트 사이트에 들어가기
        try:
            driver.get("https://www.mangoplate.com/search/{0}?keyword={0}&page={1}".format(keyword, p))
            driver.execute_script("window.scrollTo(0, 2000)")
        except:
            break

        for i in range(row):
            for j in range(col):
                # 동영상 1개 담을 그릇
                sub_dict = {}
                
                # 음식점 클릭
                title_element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > figcaption > div > a > h2'    
                point_element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > figcaption > div > strong'
                view_element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > figcaption > div > p.etc_info > span.view_count'
                review_element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > figcaption > div > p.etc_info > span.review_count'
                etc_element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > figcaption > div > p.etc'
                url_element = f'body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li:nth-child({i+1}) > div:nth-child({j+1}) > figure > figcaption > div > a'
                
                name_with_selectors = {
                    "title": title_element,
                    "point": point_element,
                    "view": view_element,
                    "review": review_element,
                    "etc": etc_element,
                    "url": url_element,
                }
                try:
                    for name, selector in name_with_selectors.items():
                        element = driver.find_element(By.CSS_SELECTOR, selector)
                        if name == "url":
                            sub_dict[name] = element.get_attribute('href')
                        elif name == "etc":
                            desc = element.text
                            loc, menu = desc.split(sep=" - ")
                            sub_dict["loc"] = loc
                            sub_dict["menu"] = menu
                        else:
                            sub_dict[name] = element.text
                except:
                    continue

                # total_dict에 담기
                total_dict[k] = sub_dict
                k += 1

    df = pd.DataFrame.from_dict(total_dict, orient='index')
    return df

In [4]:
# 크롬창 띄우기
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

In [7]:
seongsu = getFoodList(driver=driver, keyword="부산", page=50)

In [9]:
seongsu

,title,point,view,review,loc,menu,url
0,비비비당,4.7,"269,261",95,부산 해운대구,카페 / 디저트,https://www.mangoplate.com/restaurants/lNGf0ef7vT
1,할매국밥,4.6,"209,734",99,부산 동구,탕 / 찌개 / 전골,https://www.mangoplate.com/restaurants/yNGGQSGw7x
2,나가하마만게츠,4.6,"114,838",60,부산 해운대구,라멘 / 소바 / 우동,https://www.mangoplate.com/restaurants/PHhjurE...
3,톤쇼우,4.6,"103,504",57,부산 수영구,까스 요리,https://www.mangoplate.com/restaurants/TXkjVTA...
4,해목 (해운대점),4.5,"448,400",181,부산 해운대구,정통 일식 / 일반 일식,https://www.mangoplate.com/restaurants/eLq_Q72...
...,...,...,...,...,...,...,...
979,영원아이스크림바,,"2,511",3,부산 해운대구,카페 / 디저트,https://www.mangoplate.com/restaurants/-YNp3P6...
980,나인에이엠,,"2,679",2,부산 사하구,브런치 / 버거 / 샌드위치,https://www.mangoplate.com/restaurants/0SKA7JW...
981,새총횟집,,"2,598",2,부산 해운대구,해산물 요리,https://www.mangoplate.com/restaurants/wGu8MYR...
982,약속회,,"1,773",2,부산 영도구,해산물 요리,https://www.mangoplate.com/restaurants/xEvx5Vp...


In [8]:

for i in range(len(seongsu.index)):
    unique_menu.add(seongsu["menu"][i])

for i in unique_menu:
    print(i)

전통 주점 / 포차
패밀리 레스토랑
인도 음식
기타 양식
세계음식 기타
프랑스 음식
탕 / 찌개 / 전골
다국적 퓨전
기타 일식
베이커리
퓨전 중식
칵테일 / 와인
기타 중식
일반 주점
퓨전 한식
정통 중식 / 일반 중식
이자카야 / 오뎅 / 꼬치
남미 음식
태국 음식
돈부리 / 일본 카레 / 벤토
라멘 / 소바 / 우동
다국적 아시아 음식
이탈리안
퓨전 양식
닭 / 오리 요리
퓨전 일식
브런치 / 버거 / 샌드위치
기타 한식
스테이크 / 바베큐
뷔페
정통 일식 / 일반 일식
시푸드 요리
고기 요리
딤섬 / 만두
국수 / 면 요리
베트남 음식
회 / 스시
까스 요리
철판 요리
카페 / 디저트
치킨 / 호프 / 펍
해산물 요리
한정식 / 백반 / 정통 한식


In [27]:
for url in seongsu['url']:
    result = getFoodInfo(driver, url)
    print(result)

{'title': '마리오네', 'Point': 4.8, 'View': '38,517', 'Review': '23', 'Star': '921', 'Open_Hour': '', 'Breaktime': '', 'Closed_Day': '월, 화', 'Price': '만원-2만원', 'Type': '이탈리안', 'Menu': [], 'URL': 'https://www.mangoplate.com/restaurants/e5avGLO5hjJM'}
{'title': '미오도쿄다이닝', 'Point': 4.7, 'View': '118,714', 'Review': '60', 'Star': '2,163', 'Open_Hour': '11:30 - 22:00', 'Breaktime': '15:00 - 17:30', 'Closed_Day': '일, 첫째/셋째 월', 'Price': '만원-2만원', 'Type': '퓨전 일식', 'Menu': [('도쿄수제함바그고젠 (데미그라스 소스)', '13,000원'), ('도쿄수제함바그고젠 (와후 소스)', '13,000원'), ('도쿄 수제 함바그 고젠 더블 패티 (계란후라이 포함)', '21,000원'), ('치킨스테이크고젠', '11,000원'), ('포크 갈릭에 빠지다', '9,900원')], 'URL': 'https://www.mangoplate.com/restaurants/bCF4FCZPy1Cs'}
{'title': '지우관', 'Point': 4.7, 'View': '53,592', 'Review': '25', 'Star': '1,386', 'Open_Hour': '11:30 - 21:00', 'Breaktime': '15:00 - 17:00', 'Closed_Day': '월, 일', 'Price': '만원-2만원', 'Type': '딤섬 / 만두', 'Menu': [('시리얼새우', '15,000원'), ('루로우빤', '12,000원'), ('멜론하이볼', '8,000원'), ('지우관하이볼', '8,000원'), ('지우관교

In [52]:
df[:]["Type"]

0    돈부리 / 일본 카레 / 벤토
1               기타 양식
2              베트남 음식
3         치킨 / 호프 / 펍
4       정통 중식 / 일반 중식
5               기타 한식
6     브런치 / 버거 / 샌드위치
7            카페 / 디저트
Name: Type, dtype: object